In [1]:
import logging
import os
import pdb
import warnings
from pathlib import Path

import metpy.constants
import numpy as np
import pandas as pd
import seaborn as sns
import xarray
from hwt import firstRun, fv3, helicityThresholds, mpas, windThresholds
from matplotlib.colors import ListedColormap
from metpy.units import units
from sklearn.neighbors import BallTree
from tqdm import tqdm

sns.set_theme()
logging.basicConfig(level=logging.WARNING, format="%(asctime)s %(message)s", force=True)
tmpdir = Path(os.getenv("TMPDIR"))

In [2]:
def assigncoords(ds: xarray.Dataset):
    """
    Assign member number, dayForecast, forecast hour,
    initialization time, and valid time to xarray Dataset.
    - Read member number from substring component of file path.
    - Get forecast hour from global attribute (.attrs["forecastHour"])
    - Initialization time comes from global attribute `initializationTime`.
    - Valid time derived from initialization time and forecast hour.
    """

    # get member number from original name of file, which is
    # held in Dataset.encoding["source"].
    filename = Path(ds.encoding["source"])
    logging.info(filename)
    # grab part that starts with "mem"
    mem = [p for p in filename.parts if p.startswith("mem")]
    # strip off the "mem_" part and keep the reset
    mem = mem[0].lstrip("mem_")
    # convert to integer
    mem = int(mem)

    # read forecastHour from global attribute
    forecastHour = ds.attrs["forecastHour"]
    forecastHour = float(forecastHour)

    # assign dayForecast (after forecastHour is defined)
    # add 11 instead of 12 because we want forecastHours 13-36 to map to day 1
    # because uh max covers the previous hour. so does 10-m wind max
    # forecastHours 37-60 map to day 2
    dayForecast = int((forecastHour + 11) / 24)

    # read initializationTime from global attribute
    initializationTime = ds.attrs["initializationTime"]
    initializationTime = pd.to_datetime(initializationTime, format="%Y%m%d%H")

    #  valid_time = initializationTime + forecastHour
    valid_time = initializationTime + pd.to_timedelta(forecastHour, unit="hour")

    # I brought these assignments down here together to see if it is faster.
    # assign to coordinate
    ds = ds.assign_coords(mem=mem, dayForecast=dayForecast)
    # thought assign_coords would add dim to data_vars but it didn't
    ds = ds.expand_dims(dim=["mem", "dayForecast"])
    # no square brackets around initializationTime so it doesn't become a coordinate.
    # ds = ds.assign(initializationTime=initializationTime)
    ds = ds.assign(
        forecastHour=forecastHour,
        initializationTime=initializationTime,
        valid_time=[valid_time],
    )
    return ds

In [3]:
group = "wind"
thresholds = helicityThresholds if group == "uh" else windThresholds
genericnames = ["updraft max", "10m speed max"]
if group == "uh":
    genericnames = ["0-1km UH", "0-3km UH", "2-5km UH"]
fv3.group = group
mpas.group = group

models = [fv3, mpas]
# assert models all have same number of variables
nvars = len(genericnames)
assert all(
    [len(model.v) == nvars for model in models]
), "Models have different number of variables to analyze"
day1_forecast_hours = range(13, 37)
swindow = 25 # size of smoothing window in lon and lat dimensions

In [4]:
example = "/glade/campaign/mmm/parc/schwartz/HWT2024/mpas/2024052100/post/mem_1/interp_mpas_3km_2024052100_mem1_f018.nc"
ds = xarray.open_dataset(example).squeeze()  # squeeze 1-element time dimension
latitudes = ds.latitude
longitudes = ds.longitude

In [10]:
client.cluster.close()
client.shutdown()
client.close()

In [5]:
# worked well on casper batch with 32 CPUs (36 possible but took long time in queue)
# and 30G memory
from dask.distributed import Client

client = Client(n_workers=35, threads_per_worker=1)  # 70)
client

2024-11-22 09:57:40,366 State start
2024-11-22 09:57:40,424 Found stale lock file and directory '/glade/derecho/scratch/ahijevyc/tmp/dask-scratch-space/scheduler-pfcjjzhy', purging
2024-11-22 09:57:40,498   Scheduler at:     tcp://127.0.0.1:38663
2024-11-22 09:57:40,499   dashboard at:  https://jupyterhub.hpc.ucar.edu/stable/user/ahijevyc/proxy/8787/status
2024-11-22 09:57:40,499 Registering Worker plugin shuffle
2024-11-22 09:57:40,672         Start Nanny at: 'tcp://127.0.0.1:45321'
2024-11-22 09:57:40,682         Start Nanny at: 'tcp://127.0.0.1:46593'
2024-11-22 09:57:40,686         Start Nanny at: 'tcp://127.0.0.1:35935'
2024-11-22 09:57:40,689         Start Nanny at: 'tcp://127.0.0.1:38495'
2024-11-22 09:57:40,693         Start Nanny at: 'tcp://127.0.0.1:33821'
2024-11-22 09:57:40,696         Start Nanny at: 'tcp://127.0.0.1:45759'
2024-11-22 09:57:40,701         Start Nanny at: 'tcp://127.0.0.1:40773'
2024-11-22 09:57:40,704         Start Nanny at: 'tcp://127.0.0.1:40287'
2024-11

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/ahijevyc/proxy/8787/status,
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/ahijevyc/proxy/8787/status,Workers: 35
Total threads: 35,Total memory: 69.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:38663,Workers: 35
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/ahijevyc/proxy/8787/status,Total threads: 35
Started: Just now,Total memory: 69.00 GiB
Comm: tcp://127.0.0.1:44335,Total threads: 1
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/ahijevyc/proxy/40501/status,Memory: 1.97 GiB
Nanny: tcp://127.0.0.1:45321,


2024-11-22 09:58:25,017 Event loop was unresponsive in Nanny for 3.50s.  This is often caused by long-running GIL-holding functions or moving large chunks of data. This can cause timeouts and instability.
2024-11-22 09:58:26,144 Event loop was unresponsive in Nanny for 4.62s.  This is often caused by long-running GIL-holding functions or moving large chunks of data. This can cause timeouts and instability.
2024-11-22 09:58:26,661 Event loop was unresponsive in Nanny for 5.14s.  This is often caused by long-running GIL-holding functions or moving large chunks of data. This can cause timeouts and instability.
2024-11-22 09:58:27,791 Event loop was unresponsive in Nanny for 6.27s.  This is often caused by long-running GIL-holding functions or moving large chunks of data. This can cause timeouts and instability.
2024-11-22 09:58:28,041 Event loop was unresponsive in Nanny for 6.52s.  This is often caused by long-running GIL-holding functions or moving large chunks of data. This can cause t

In [ ]:
d2023 = pd.date_range(start=firstRun(2023), end="20230531", freq="1D")
d2024 = pd.date_range(start=firstRun(2024), end="20240531", freq="1D")
valid_dates = d2023.union(d2024)

for valid_date in valid_dates:
    rptfile = tmpdir / f"near_rpt.{valid_date}.nc"
    if os.path.exists(rptfile):
        logging.warning(f"open existing {rptfile}")
        near_rpts = xarray.open_dataarray(rptfile)
    else:
        logging.warning(f"create new {rptfile}")
        # Load severe weather reports
        near_rpts = []
        rpt_types = ["torn", "wind", "hail"]
        for rpt_type in rpt_types:
            rpt_dist_thresh_miles = 25 * units.miles
            reports = pd.read_csv(
                f"https://www.spc.noaa.gov/climo/reports/{valid_date.strftime('%y%m%d')}_rpts_{rpt_type}.csv"
            )
            print(f"read {len(reports)} {rpt_type} reports {valid_date}")

            logging.info("Create DataArray like latitudes with all elements set to False")
            near_rpt = xarray.full_like(latitudes, False, dtype=bool)
            near_rpt.name = rpt_type
            del near_rpt.attrs['units']
            del near_rpt.attrs['long_name']
            near_rpt.attrs["range"] = str(rpt_dist_thresh_miles)

            # If at least one report, put True in neighboring values
            if not reports.empty:
                # Prepare wind report coordinates for spatial indexing
                rpt_coords = reports[["Lat", "Lon"]]

                # Prepare model grid coordinates for spatial indexing
                # .ravel reads 2-d array as if it were a 1-d array.
                uh_coords = np.stack(latitudes.values.ravel(), longitudes.values.ravel()).T

                # BallTree (with metric = "haversine") assumes spherical coordinates
                uh_tree = BallTree(np.deg2rad(uh_coords), metric="haversine")

                Re = metpy.constants.earth_avg_radius
                r = rpt_dist_thresh_miles.to("km") / Re
                r = r.to_base_units()
                # Find all uh points whose distance is at most threshold_distance from wind report
                results = uh_tree.query_radius(np.deg2rad(rpt_coords), r=r)
                for result in results:
                    near_rpt.values.put(result, True)

            near_rpts.append(near_rpt)
        # tried concat with dim argument but it didn't preserve coord labels
        near_rpts = xarray.merge(near_rpts).to_dataarray(dim="rpt_type", name="near_rpts")
        near_rpts.to_netcdf(rptfile)
    for model in models:
        idir = Path(f"/glade/campaign/mmm/parc/schwartz/HWT{valid_date.strftime('%Y')}/{model}")
        ncfile = (
            tmpdir
            / f"forecast_yes.{model}.{thresholds.attrs['short_name']}.{swindow}.{valid_date.strftime('%Y%m%d')}.nc"
        )
        if os.path.exists(ncfile) and (os.stat(ncfile).st_size > 50000):
            logging.warning(f"skip existing {ncfile}")
            continue
        else:
            logging.warning(f"create new {ncfile}")
            # Load updraft helicity data from netCDF
            fmt = "%Y%m%d%H"
            oneday = pd.to_timedelta(1, unit="day")

            # Create list of input files
            # This is a nested list comprehension, looping through
            # day1_forecast_hours (iterable of forecast hours)
            #    Model.lead_time_days (iterable of lead times in days)
            #        members (1 through max_members)
            max_members = 10
            ifiles = [
                idir
                / (valid_date - lead_time_day * oneday).strftime(fmt)
                / "post"
                / f"mem_{mem}"
                / f"interp_{model}_3km_{(valid_date-lead_time_day*oneday).strftime(fmt)}_mem{mem}_f{fhr+lead_time_day*24:03d}.nc"
                for mem in range(1, max_members + 1)
                for lead_time_day in range(model.lead_time_days)
                for fhr in day1_forecast_hours
            ]
            ifiles = [f for f in ifiles if os.path.exists(f)]
            print(f"open {len(ifiles)} files")
            print(ifiles[0:8])
            assert len(ifiles) % 24 == 0, "# ifiles should be multiple of 24"
            ds = xarray.open_mfdataset(
                ifiles,
                preprocess=assigncoords,
                drop_variables=["total_precip_hrly"],
                combine_attrs="drop",
                compat="override",
                coords="minimal",
                parallel=True,
                decode_cf=False,
                decode_coords=False,
                engine="h5netcdf",  # "h5netcdf" helped with dask and HDF/lock errors
            )

            # extract model variables and remove time dimension
            ds = ds[model.v].squeeze(dim="time", drop=True)
        
            # Take the maximum along the valid_time dimension.
            # This is the 24-hour ensemble maximum. It is a 24-hour max because
            # we extracted slice `day1_forecast_hours` plus a multiple of 24
            # hours. Don't forget the dayForecast dimension. Each file contains
            # multiple forecasts of different lead times valid for the same
            # 24-hour period.
            # Each file contains the day-1, day-2, ..., day-Model.lead_time_day
            # forecasts.
            logging.warning("24-h max")
            ds = ds.max(dim=["valid_time"])
        
            # Combine multiple DataArrays associated with group "uh" or "wind"
            # into a single DataArray with a new "variable" dimension.
            da = ds.to_dataarray()
            
            
            center = True
            logging.warning(f"spatial smooth {swindow} {center}")
            da = da.rolling(lat=swindow, lon=swindow, min_periods=1, center=center).max()
        
            nmem = da.notnull().sum(dim="mem")
            # Count the number of members that exceed the threshold.
            # This count divided by the number of members nmem is the
            # ensemble probability.
            logging.warning(f"enssum {len(thresholds)} thresh")
            da = xarray.concat(
                [(da >= t).sum(dim="mem") for t in thresholds],
                dim="thresh",
            ).assign_coords(thresh=thresholds)
            da.name = "enssum"
            da.attrs["rolling_spatial_window"] = swindow
            da.attrs["model"] = str(model)
            da["nmem"] = nmem
        
            da.to_netcdf(ncfile)
        
            logging.warning(f"saved {ncfile}")

2024-11-22 09:57:54,283 open existing /glade/derecho/scratch/ahijevyc/tmp/near_rpt.2023-04-24 00:00:00.nc
2024-11-22 09:57:54,324 skip existing /glade/derecho/scratch/ahijevyc/tmp/forecast_yes.fv3.wind.25.20230424.nc
2024-11-22 09:57:54,325 skip existing /glade/derecho/scratch/ahijevyc/tmp/forecast_yes.mpas.wind.25.20230424.nc
2024-11-22 09:57:54,326 open existing /glade/derecho/scratch/ahijevyc/tmp/near_rpt.2023-04-25 00:00:00.nc
2024-11-22 09:57:54,362 skip existing /glade/derecho/scratch/ahijevyc/tmp/forecast_yes.fv3.wind.25.20230425.nc
2024-11-22 09:57:54,364 skip existing /glade/derecho/scratch/ahijevyc/tmp/forecast_yes.mpas.wind.25.20230425.nc
2024-11-22 09:57:54,364 open existing /glade/derecho/scratch/ahijevyc/tmp/near_rpt.2023-04-26 00:00:00.nc
2024-11-22 09:57:54,396 skip existing /glade/derecho/scratch/ahijevyc/tmp/forecast_yes.fv3.wind.25.20230426.nc
2024-11-22 09:57:54,397 skip existing /glade/derecho/scratch/ahijevyc/tmp/forecast_yes.mpas.wind.25.20230426.nc
2024-11-22 09

open 600 files
[PosixPath('/glade/campaign/mmm/parc/schwartz/HWT2023/mpas/2023051800/post/mem_1/interp_mpas_3km_2023051800_mem1_f013.nc'), PosixPath('/glade/campaign/mmm/parc/schwartz/HWT2023/mpas/2023051800/post/mem_1/interp_mpas_3km_2023051800_mem1_f014.nc'), PosixPath('/glade/campaign/mmm/parc/schwartz/HWT2023/mpas/2023051800/post/mem_1/interp_mpas_3km_2023051800_mem1_f015.nc'), PosixPath('/glade/campaign/mmm/parc/schwartz/HWT2023/mpas/2023051800/post/mem_1/interp_mpas_3km_2023051800_mem1_f016.nc'), PosixPath('/glade/campaign/mmm/parc/schwartz/HWT2023/mpas/2023051800/post/mem_1/interp_mpas_3km_2023051800_mem1_f017.nc'), PosixPath('/glade/campaign/mmm/parc/schwartz/HWT2023/mpas/2023051800/post/mem_1/interp_mpas_3km_2023051800_mem1_f018.nc'), PosixPath('/glade/campaign/mmm/parc/schwartz/HWT2023/mpas/2023051800/post/mem_1/interp_mpas_3km_2023051800_mem1_f019.nc'), PosixPath('/glade/campaign/mmm/parc/schwartz/HWT2023/mpas/2023051800/post/mem_1/interp_mpas_3km_2023051800_mem1_f020.nc')]


2024-11-22 09:58:40,051 24-h max
2024-11-22 09:58:40,077 spatial smooth 25 True
2024-11-22 09:58:40,205 enssum 10 thresh
2024-11-22 09:59:16,261 saved /glade/derecho/scratch/ahijevyc/tmp/forecast_yes.mpas.wind.25.20230518.nc
2024-11-22 09:59:16,263 open existing /glade/derecho/scratch/ahijevyc/tmp/near_rpt.2023-05-19 00:00:00.nc
2024-11-22 09:59:16,656 create new /glade/derecho/scratch/ahijevyc/tmp/forecast_yes.fv3.wind.25.20230519.nc


open 1920 files
[PosixPath('/glade/campaign/mmm/parc/schwartz/HWT2023/fv3/2023051900/post/mem_1/interp_fv3_3km_2023051900_mem1_f013.nc'), PosixPath('/glade/campaign/mmm/parc/schwartz/HWT2023/fv3/2023051900/post/mem_1/interp_fv3_3km_2023051900_mem1_f014.nc'), PosixPath('/glade/campaign/mmm/parc/schwartz/HWT2023/fv3/2023051900/post/mem_1/interp_fv3_3km_2023051900_mem1_f015.nc'), PosixPath('/glade/campaign/mmm/parc/schwartz/HWT2023/fv3/2023051900/post/mem_1/interp_fv3_3km_2023051900_mem1_f016.nc'), PosixPath('/glade/campaign/mmm/parc/schwartz/HWT2023/fv3/2023051900/post/mem_1/interp_fv3_3km_2023051900_mem1_f017.nc'), PosixPath('/glade/campaign/mmm/parc/schwartz/HWT2023/fv3/2023051900/post/mem_1/interp_fv3_3km_2023051900_mem1_f018.nc'), PosixPath('/glade/campaign/mmm/parc/schwartz/HWT2023/fv3/2023051900/post/mem_1/interp_fv3_3km_2023051900_mem1_f019.nc'), PosixPath('/glade/campaign/mmm/parc/schwartz/HWT2023/fv3/2023051900/post/mem_1/interp_fv3_3km_2023051900_mem1_f020.nc')]


2024-11-22 10:01:43,515 24-h max
2024-11-22 10:01:43,589 spatial smooth 25 True
2024-11-22 10:01:43,913 enssum 10 thresh
/glade/u/apps/opt/conda/envs/npl-2024b/lib/python3.11/site-packages/distributed/client.py:3245: UserWarning: Sending large graph of size 15.01 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(
2024-11-22 10:03:42,256 saved /glade/derecho/scratch/ahijevyc/tmp/forecast_yes.fv3.wind.25.20230519.nc
2024-11-22 10:03:42,258 create new /glade/derecho/scratch/ahijevyc/tmp/forecast_yes.mpas.wind.25.20230519.nc


...

2024-11-22 11:54:34,716 24-h max
2024-11-22 11:54:34,742 spatial smooth 25 True
2024-11-22 11:54:34,946 enssum 10 thresh
2024-11-22 11:55:12,374 saved /glade/derecho/scratch/ahijevyc/tmp/forecast_yes.mpas.wind.25.20240429.nc
2024-11-22 11:55:12,380 open existing /glade/derecho/scratch/ahijevyc/tmp/near_rpt.2024-04-30 00:00:00.nc
2024-11-22 11:55:12,734 create new /glade/derecho/scratch/ahijevyc/tmp/forecast_yes.fv3.wind.25.20240430.nc


open 1920 files
[PosixPath('/glade/campaign/mmm/parc/schwartz/HWT2024/fv3/2024043000/post/mem_1/interp_fv3_3km_2024043000_mem1_f013.nc'), PosixPath('/glade/campaign/mmm/parc/schwartz/HWT2024/fv3/2024043000/post/mem_1/interp_fv3_3km_2024043000_mem1_f014.nc'), PosixPath('/glade/campaign/mmm/parc/schwartz/HWT2024/fv3/2024043000/post/mem_1/interp_fv3_3km_2024043000_mem1_f015.nc'), PosixPath('/glade/campaign/mmm/parc/schwartz/HWT2024/fv3/2024043000/post/mem_1/interp_fv3_3km_2024043000_mem1_f016.nc'), PosixPath('/glade/campaign/mmm/parc/schwartz/HWT2024/fv3/2024043000/post/mem_1/interp_fv3_3km_2024043000_mem1_f017.nc'), PosixPath('/glade/campaign/mmm/parc/schwartz/HWT2024/fv3/2024043000/post/mem_1/interp_fv3_3km_2024043000_mem1_f018.nc'), PosixPath('/glade/campaign/mmm/parc/schwartz/HWT2024/fv3/2024043000/post/mem_1/interp_fv3_3km_2024043000_mem1_f019.nc'), PosixPath('/glade/campaign/mmm/parc/schwartz/HWT2024/fv3/2024043000/post/mem_1/interp_fv3_3km_2024043000_mem1_f020.nc')]


2024-11-22 11:56:39,198 full garbage collections took 10% CPU time recently (threshold: 10%)
2024-11-22 11:56:43,045 full garbage collections took 10% CPU time recently (threshold: 10%)
2024-11-22 11:56:47,631 full garbage collections took 10% CPU time recently (threshold: 10%)
2024-11-22 11:56:52,460 full garbage collections took 10% CPU time recently (threshold: 10%)
2024-11-22 11:56:57,743 full garbage collections took 10% CPU time recently (threshold: 10%)
2024-11-22 11:57:03,824 full garbage collections took 10% CPU time recently (threshold: 10%)
2024-11-22 11:57:10,836 full garbage collections took 10% CPU time recently (threshold: 10%)
2024-11-22 11:57:39,474 24-h max
2024-11-22 11:57:39,535 spatial smooth 25 True
2024-11-22 11:57:39,796 enssum 10 thresh
2024-11-22 11:57:43,118 full garbage collections took 10% CPU time recently (threshold: 10%)
2024-11-22 11:57:48,089 full garbage collections took 10% CPU time recently (threshold: 10%)
2024-11-22 11:57:54,321 full garbage colle

open 600 files
[PosixPath('/glade/campaign/mmm/parc/schwartz/HWT2024/mpas/2024043000/post/mem_1/interp_mpas_3km_2024043000_mem1_f013.nc'), PosixPath('/glade/campaign/mmm/parc/schwartz/HWT2024/mpas/2024043000/post/mem_1/interp_mpas_3km_2024043000_mem1_f014.nc'), PosixPath('/glade/campaign/mmm/parc/schwartz/HWT2024/mpas/2024043000/post/mem_1/interp_mpas_3km_2024043000_mem1_f015.nc'), PosixPath('/glade/campaign/mmm/parc/schwartz/HWT2024/mpas/2024043000/post/mem_1/interp_mpas_3km_2024043000_mem1_f016.nc'), PosixPath('/glade/campaign/mmm/parc/schwartz/HWT2024/mpas/2024043000/post/mem_1/interp_mpas_3km_2024043000_mem1_f017.nc'), PosixPath('/glade/campaign/mmm/parc/schwartz/HWT2024/mpas/2024043000/post/mem_1/interp_mpas_3km_2024043000_mem1_f018.nc'), PosixPath('/glade/campaign/mmm/parc/schwartz/HWT2024/mpas/2024043000/post/mem_1/interp_mpas_3km_2024043000_mem1_f019.nc'), PosixPath('/glade/campaign/mmm/parc/schwartz/HWT2024/mpas/2024043000/post/mem_1/interp_mpas_3km_2024043000_mem1_f020.nc')]


2024-11-22 11:59:58,779 full garbage collections took 10% CPU time recently (threshold: 10%)


In [ ]:
ls -li /glade/derecho/scratch/ahijevyc/tmp/forecast_yes.mpas.uh.20240510.nc_bad